<a href="https://colab.research.google.com/github/stormyCobra1199/showcase/blob/main/resaleHDBpx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HDB Resale Transaction bot - by M. W. Ho (c) 2022


In [1]:
!pip install python-telegram-bot
!pip install tabulate
import pandas as pd
import requests
import telegram
import datetime
import os
import logging

from myconf import TOKEN

from tabulate import tabulate
from pandas.plotting import table 

from flask import Flask, request

from telegram import Bot, Update
from telegram.ext import Dispatcher, Filters, MessageHandler, CommandHandler, CallbackContext

from telegram.ext import Updater

updater = Updater(token=TOKEN, use_context=True)
dispatcher = updater.dispatcher

PORT = int(os.environ.get('PORT', 5000))
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

bot = telegram.Bot(token=TOKEN)
print("\n",bot.get_me(),"\n")


 {'id': 5218309496, 'first_name': 'HDBResaleTx', 'can_join_groups': True, 'is_bot': True, 'supports_inline_queries': False, 'can_read_all_group_messages': False, 'username': 'HDBResaleTx_bot'} 



In [2]:
###
def start(update: Update, context: CallbackContext):
    outF = open("trackStart.txt", "a")
    outF.write("1 ")
    outF.close()
    context.bot.send_message(chat_id=update.effective_chat.id, text="HDBRT Bot - Get latest HDB Resale Transactions.\nExamples: /hs 5 Bedok for 5Rm in Bedok or /hs French for all French Road.")
   
def hs(update: Update, context: CallbackContext):
    q=' '.join(context.args)
    if q[0] in ["2","3","4","5"]:
        url = "https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&fields=month,block,street_name,flat_type,flat_model,resale_price,floor_area_sqm,lease_commence_date,storey_range&sort=month%20desc&limit=20&filters={\"flat_type\":\""+q[0]+" ROOM"+"\",\"town\":\""+q[2:].upper()+"\"}"
        #context.bot.send_message(chat_id=update.effective_chat.id, text=url)
    else:
        url = "https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&fields=month,block,street_name,flat_type,flat_model,resale_price,floor_area_sqm,lease_commence_date,storey_range&sort=month%20desc&limit=20&q="+q
    response=requests.get(url)
    dataset=response.json()
    df = pd.json_normalize(dataset['result'], "records")  
    df = df.sort_values(by='block').set_index('block').reindex(columns=['street_name','flat_type','flat_model','resale_price','floor_area_sqm','lease_commence_date','storey_range','month'])
    df.rename(columns={'street_name': 'street', 'flat_type': 'type', 'resale_price': 'price', 'lease_commence_date': 'built', 'storey_range': 'floor', 'floor_area_sqm': 'sqm', 'flat_model': 'model', 'month': 'sold'}, inplace=True)
    df.replace(['2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'New Generation', 'Standard', 'EXECUTIVE', 'Maisonette', 'Simplified', 'Improved'],['2','3','4','5','NG','Std','E','M','S','I'], inplace=True) 
    df['type']=df['type']+df['model']
    df.drop('model', axis=1, inplace=True)
    dft=tabulate(df, headers='keys', tablefmt='plain', numalign='left')
    context.bot.send_message(chat_id=update.effective_chat.id, text=dft)
    ct = datetime.datetime.now()
    outF = open(str(ct), "w")
    outF.write(dft)
    outF.close()

def errorhand(update: Update, context: CallbackContext):
    context.bot.send_message(chat_id=update.effective_chat.id, text="Please enter a valid town or search term. Eg: /hs 5 Bedok for 5Rm in Bedok or /hs French for all French Road.")
   

In [ ]:
###
app = Flask(__name__)
@app.route("/")
def main():
  start_handler = CommandHandler('start', start)
  dispatcher.add_handler(start_handler)

  hs_handler = CommandHandler('hs', hs)
  dispatcher.add_handler(hs_handler)
  dispatcher.add_error_handler(errorhand)

  bot.send_message(text='Bot initiated.', chat_id=1349303768)

  #####
  #updater.start_polling()
  updater.start_webhook(listen="0.0.0.0",
                          port=int(PORT),
                          url_path=TOKEN)
  updater.bot.setWebhook('https://rocky-lowlands-28256.herokuapp.com/' + TOKEN)


if __name__ == "__main__":
    main()
    #app.run(debug=True, host="0.0.0.0", port=int(os.environ.get("PORT", 8080)))

### run in Console F12

function KeepClicking(){
console.log("Clicking");
document.querySelector("colab-connect-button").click() }
setInterval(KeepClicking,60000)

### for keeping alive

import time
t_end = time.time() + 200000 
while time.time() < t_end:
    time.sleep(360)
    print("Stayin' alive...")

### for testing API and response only

### testing return image
import matplotlib.pyplot as plt
ax = plt.subplot(frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis
table(ax, df)  # where df is your data frame
plt.rcParams["figure.figsize"] = (18, 1)
plt.savefig('mytable.png')